In [ ]:
import pandas as pd
import json
from sqlalchemy import create_engine

db_url = "mysql+pymysql://root:@localhost/keranjang_homemade"
engine = create_engine(db_url)

# Query untuk online sales
query_online = """
SELECT 
    DATE_FORMAT(order_date, '%%Y-%%m') AS month,
    SUM(total_amount) AS online_sales
FROM orders
WHERE status = 'Completed'
GROUP BY month
ORDER BY month;
"""

# Query untuk offline sales
query_offline = """
SELECT 
    DATE_FORMAT(order_date, '%%Y-%%m') AS month,
    SUM(total_amount) AS offline_sales
FROM offline_orders
WHERE status = 'Completed'
GROUP BY month
ORDER BY month;
"""

# Query untuk mengambil daftar tahun yang ada di database
query_years = """
SELECT DISTINCT YEAR(order_date) AS year
FROM orders
WHERE status = 'Completed'
ORDER BY year DESC;
"""

# Ambil data dari database
df_online = pd.read_sql(query_online, engine)
df_offline = pd.read_sql(query_offline, engine)
df_years = pd.read_sql(query_years, engine)

# Ambil daftar tahun
years = df_years['year'].tolist()

# Gabungkan data online dan offline berdasarkan bulan
df = pd.merge(df_online, df_offline, on='month', how='outer')

# Tangani nilai yang hilang (NaN) dengan menggantinya dengan 0
df = df.fillna(0)

# Format data bulan menjadi nama bulan
df['month'] = pd.to_datetime(df['month'], format='%Y-%m').dt.strftime('%Y-%B')
df['total_sales'] = df['online_sales'] + df['offline_sales']

# Format data menjadi JSON
data = {
    "labels": df["month"].tolist(),
    "online_sales": df["online_sales"].tolist(),
    "offline_sales": df["offline_sales"].tolist(),
    "total_sales": df["total_sales"].tolist(),
    "years": years
}

# Simpan ke file JSON
with open('C:\\laragon\\www\\keranjang_homemade\\graphics\\sales_data.json', 'w') as f:
    json.dump(data, f)